In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 

import matplotlib.pyplot as plt

In [ ]:
files = []
y_list = []

# load the binary files and add target variables
for class_n, c in enumerate(['kiss', 'handshake', 'hug', 'highfive']):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))

    # only get first 50 samples (so not to interfere with augmented samples)
    filtered.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))
    filtered = filtered[:50]

    y_list.extend([class_n]*len(filtered))
    files.extend(filtered)

y = np.array(y_list)

# scale data to [0, 1] range
x = [np.load(x)/255 for x in files]

In [ ]:
print(x[0].dtype)
assert len(x) == len(y)

In [ ]:
def split(X, y, test_size=0.1):
    shuffled_idx = np.random.permutation(len(X))
    
    split_at = int(len(shuffled_idx) * test_size)
    train_idx = shuffled_idx[split_at:]
    test_idx = shuffled_idx[:split_at]
    
    #TODO: add use of one-hot encoding
    onehot = np.zeros((y.size, 4))
    onehot[np.arange(y.size),y] = 1
    
    train_X = itemgetter(*train_idx)(X)
    train_y = itemgetter(*train_idx)(y)
    
    test_X = itemgetter(*test_idx)(X)
    test_y = itemgetter(*test_idx)(y)
    
    return (train_X, test_X, train_y, test_y)

In [ ]:
train_X, test_X, train_y, test_y = split(x, y)

assert len(train_X) == len(train_y)

In [ ]:
epochs = 60
shuffled = list(zip(x, y))
shuffle(shuffled)

def batcherize(df):
    for _ in range(epochs*10):
        for features, labels in shuffled:
            _features = np.array([features])
            yield _features, np.atleast_1d(labels)
        
batcherize_train = batcherize(shuffled[:190])

def batcherize_t():
    for features, labels in shuffled[190:]:
        _features = np.array([features])
        yield _features, np.atleast_1d(labels)

In [ ]:
test_df = tf.data.Dataset.from_generator(batcherize_t, output_types=(tf.float64, tf.int64))

In [ ]:
h = model.fit(batcherize_train, batch_size=1, verbose=1, epochs=epochs, steps_per_epoch=190, validation_data=test_df, validation_steps=10)

In [ ]:
plt.plot(h.history['loss'], label='Training')
plt.plot(h.history['val_loss'], label='Test')
plt.legend()

In [ ]:
plt.plot(h.history['accuracy'], label='Training')
plt.plot(h.history['val_accuracy'], label='Test')
plt.legend()